In [ ]:
reset -fs

In [121]:
# import libraries
import requests
import pandas as pd
import numpy as np
import time
import sqlite3
from bs4 import BeautifulSoup

In [46]:
# Chloe's NY Times api key

nyt_api_key = 'oblnEzHIOX44LSX4J6zY6R4tw8nCmEFi'

In [338]:
# Keaton's NY times api key

keaton_nyt_api_key = 'UZ3q47QX6i6G2TtlGZSlfbgaWaS8iJXK'

The article search code below came from here: https://towardsdatascience.com/scraping-news-and-articles-from-public-apis-with-python-be84521d85b9

Another good-looking resource: https://towardsdatascience.com/getting-newspaper-article-data-and-saving-it-to-a-sql-database-651602ee929f

# "Modern Love" query

In [481]:
modern_love_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=kicker%3A(%22Modern%20Love%22)&sort=newest&api-key=oblnEzHIOX44LSX4J6zY6R4tw8nCmEFi'

# extract results from the first page
modern_love = requests.get(modern_love_url).json() 

In [482]:
# get the number of hits (results) from the query
modern_love_hits = modern_love['response']['meta']['hits']

# get the number of pages' worth of hits
modern_love_max_pages = np.round((modern_love_hits / 10)-1,0)
print(modern_love_hits,modern_love_max_pages)

1174 116.0


In [483]:
# create urls for all pages from "Modern Love" query to loop through to extract pertinent articles

base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=kicker%3A(%22Modern%20Love%22)'
middle_url = '&page='
end_url = '&sort=newest&api-key=UZ3q47QX6i6G2TtlGZSlfbgaWaS8iJXK'

modern_love_pages = ['https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=kicker%3A(%22Modern%20Love%22)&sort=newest&api-key=oblnEzHIOX44LSX4J6zY6R4tw8nCmEFi']

for i in range(1, int(modern_love_max_pages)+1):
    page_url = (''.join(base_url + '&page=' + str(i) + end_url))
    modern_love_pages.append(page_url)

In [484]:
len(modern_love_pages)

117

### Get first 100 pages' worth of results

In [485]:
# code to get results from the first 100 pages (since that's the limit)
modern_love_pages_results = []

for p in modern_love_pages[:100]:
    results = requests.get(p).json()
    modern_love_pages_results.append(results)
    time.sleep(6)

In [486]:
len(modern_love_pages_results)

100

### Get the rest of the pages' worth of results

In [489]:
# code to get results from the rest of the pages
modern_love_pages_results_2 = []

for p in modern_love_pages[100:]:
    results = requests.get(p).json()
    modern_love_pages_results_2.append(results)
    time.sleep(6)

In [490]:
# create function for extracting ['response']['docs'] from each result on a page to get the articles
def get_articles(results_list):
    
    articles_list = []
    
    for i in range(0,len(results_list)):
        try:
            article = results_list[i]['response']['docs']
            articles_list.append(article)
        except KeyError:
            article = 0
            articles_list.append(article)
    
    return articles_list

In [491]:
len(modern_love_pages_results)

100

In [492]:
len(modern_love_pages_results_2)

17

In [493]:
#combine lists of pages results
joined_list = modern_love_pages_results + modern_love_pages_results_2

In [494]:
joined_list_articles = get_articles(joined_list)

In [495]:
len(joined_list_articles)

117

In [497]:
test = [x for x in joined_list_articles if x!= 0]

In [498]:
len(test)

117

In [426]:
joined_list_articles[0]

[{'abstract': 'In a special collaboration between Modern Love and Black History, Continued, we gathered stories that illuminate how Black people live, and love, in this moment.',
  'web_url': 'https://www.nytimes.com/2022/02/11/style/what-is-black-love-today.html',
  'snippet': 'In a special collaboration between Modern Love and Black History, Continued, we gathered stories that illuminate how Black people live, and love, in this moment.',
  'lead_paragraph': 'When we invited readers — as well as renowned writers such as Staceyann Chin, Mahogany Browne and Damon Young — to respond to the question “What is Black love today?” we were moved by the strength, resilience and diversity of the stories they told us.',
  'print_section': 'ST',
  'print_page': '6',
  'source': 'The New York Times',
  'multimedia': [{'rank': 0,
    'subtype': 'xlarge',
    'caption': None,
    'credit': None,
    'type': 'image',
    'url': 'images/2022/02/08/multimedia/BlackModernLove-Cover/BlackModernLove-02-art

In [520]:
# remove the list within the list of modern_love_articles for easier parsing
joined_list_articles_flat = [item for sublist in joined_list_articles for item in sublist]

In [650]:
# build the dataframe
articles_df = pd.DataFrame(columns=['article_id', 'article_url', 'article_headline', 'article_abstract', 
                                    'article_snippet', 'article_lead_paragraph', 'article_pub_date',
                                    'article_news_desk', 'article_section', 'article_type_material',
                                    'article_word_count'])

In [651]:
# code for extracting a piece of data about each article (aka dictionary) in the list of article dictionaries
for article in joined_list_articles_flat:
    
    article_id = article['_id']
    article_url = article['web_url']
    article_headline = article['headline']['main']
    article_abstract = article['abstract']
    article_snippet = article['snippet']
    article_lead_paragraph = article['lead_paragraph']
    article_pub_date = article['pub_date']
    article_news_desk = article['news_desk']
    article_section = article['section_name']
    article_type_material = article['type_of_material']
    article_word_count = article['word_count']
    
    # save data in articles_df
    articles_df = articles_df.append({'article_id':article_id, 'article_url':article_url,
                                     'article_headline':article_headline, 'article_abstract':article_abstract,
                                     'article_snippet':article_snippet,'article_lead_paragraph':article_lead_paragraph,
                                     'article_pub_date':article_pub_date,'article_news_desk':article_news_desk,
                                     'article_section':article_section,'article_type_material':article_type_material,
                                     'article_word_count':article_word_count},ignore_index=True)

In [652]:
articles_df.shape

(1170, 11)

In [653]:
# save dataframe
articles_df.to_csv('modern_love_articles.csv',index=False)

# Parse Modern Love in Miniature page

In [123]:
ml_in_miniature_url = 'https://www.nytimes.com/2022/02/15/style/tiny-modern-love-stories-never-good-enough-never-skinny-enough.html'

In [124]:
response = requests.get(ml_in_miniature_url)

In [323]:
soup = BeautifulSoup(response.text, "lxml")

In [324]:
soup.find_all('p',class_="css-axufdj evys1bk0")

[<p class="css-axufdj evys1bk0">My husband wears an expensive anti-snoring device. “Guh-nigh,” he says, mumbling through the mouthpiece — and discomfort — as we settle into bed. His bone-rattling inhalations used to set my teeth on edge. Now, only occasional “breakthrough snores” reach me. When I catch glimpses of the monstrosity in his mouth, I remember what he endures for my rest. In turn, I practice responding more gently when he irks me in daylight hours (“Always walking ahead of me on hikes? That’s all right!”). The contraption is meant to improve our sleep, but the real change is our newfound grace when awake. — <em class="css-2fg4z9 e1gzwzxm0">Melissa Grego</em></p>,
 <p class="css-axufdj evys1bk0">For most of my life, I feared that I was never doing enough, never good enough, never skinny enough. This surprises people who know me as a strong, positive woman, a generous teacher and mother of two sons. Alas, it took a diagnosis of Stage 4 pancreatic cancer — the incurable, termin

# Parse Modern Love page

In [325]:
ml_test_article = 'https://www.nytimes.com/2022/01/14/style/modern-love-kinder-gentler-divorce.html'

response = requests.get(ml_test_article)

In [326]:
soup = BeautifulSoup(response.text, "lxml")

In [328]:
paragraphs = soup.find_all('p',class_="css-axufdj evys1bk0")

In [336]:
len(paragraphs)

32

In [337]:
count_paragraphs = 0

for p in paragraphs:
    print(p.text)

When my ex-husband’s girlfriend stepped out of the bathroom wrapped in a towel, beads of water dripping from her brown hair, she ran into me, the ex-wife, dashing from the bedroom they often share, with my ex-husband’s dirty clothes in my arms.
“Hi, I was just getting his — ” I said before scurrying back downstairs, where I was doing our laundry.
I can think of few moments that better capture that time in our lives: me with my ex’s pungent laundry in my arms, trying to disappear as if I were the maid to a volatile celebrity.
For two people who need a prefix of negation to refer to each other, my ex and I have had a rather porous boundary between my place and his. He and I live on separate floors of a two-family house in Brooklyn. Our 8-year-old son can run upstairs to beg his father to let him play Minecraft and run downstairs to have the Cheerios he likes with me. I dip into my ex’s apartment when a recipe calls for chia seeds, and he knocks on my door when I need help resetting the c